This time try a deep network with 5 fully connected layers.

In [ ]:
import tensorflow as tf

### Training data placeholder

In [ ]:
# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X = tf.placeholder(tf.float32, [None, 28, 28, 1])

### Define the number of neurons in each layer

In [ ]:
# five layers and their number of neurons
L = 200
M = 100
N = 60
O = 30

### Variables for weights and biases

#### Tasks:
 1. How many neurons should be in the last (softmax) layer?
 1. Seed the weights with random values to break the symmetry of the neurons. You can use the [tf.truncated_normal(shape, mean=0.0, stddev=1.0)](https://www.tensorflow.org/api_docs/python/tf/truncated_normal) function to get random values from a truncated normal distribution. A standard deviation of 0.1 will work.

In [ ]:
# Weights initialised with small random values between -0.2 and +0.2
# hint - start with tensor dimensions (image w * h and the number of neurons in your first layer)
W1 = tf.Variable(#TODO)
B1 = tf.Variable(tf.zeros([L]))
W2 = tf.Variable(#TODO)
B2 = tf.Variable(tf.zeros([M]))
W3 = tf.Variable(#TODO)
B3 = tf.Variable(tf.zeros([N]))
W4 = tf.Variable(#TODO)
B4 = tf.Variable(tf.zeros([O]))
W5 = tf.Variable(#TODO)

B5 = tf.Variable(tf.zeros([#TODO]))

### Define the model
#### Tasks:
 1. Define your model. This time we will use the [sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) activation function. See [tf.nn.sigmoid(x, name=None)](https://www.tensorflow.org/api_docs/python/tf/sigmoid).
 2. Which function will you use in the last layer to get your classification as a probability?

In [ ]:
# The model
XX = tf.reshape(X, [-1, 784])
# TODO: define 4 layers here
Ylogits = tf.matmul(Y4, W5) + B5
Y = tf.nn.#TODO(Ylogits)

# correct answers will go here
Y_ = tf.placeholder(tf.float32, [None, 10])

### Define the error function

In [ ]:
# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
# TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability
# problems with log(0) which is NaN
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)*100

### Concatenate the weights and biases

In [ ]:
allweights = tf.concat([tf.reshape(W1, [-1]), tf.reshape(W2, [-1]), tf.reshape(W3, [-1]), tf.reshape(W4, [-1]), tf.reshape(W5, [-1])], 0)
allbiases  = tf.concat([tf.reshape(B1, [-1]), tf.reshape(B2, [-1]), tf.reshape(B3, [-1]), tf.reshape(B4, [-1]), tf.reshape(B5, [-1])], 0)

### Keep track of the prediction accuracy

In [ ]:
# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Configure the optimizer

In [ ]:
# training step, learning rate = 0.003
learning_rate = 0.003
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

### Load the training and test data

In [ ]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets

# Download images and labels into mnist.test (10K images+labels) and mnist.train (60K images+labels)
mnist = read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

### A training function to iterate over

In [ ]:
# Initialise the variables
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

test_data = {X: mnist.test.images, Y_: mnist.test.labels}

# You can call this function in a loop to train the model, 100 images at a time
def training_step(i, update_test_data, update_train_data):

    # training on batches of 100 images with 100 labels
    batch_X, batch_Y = mnist.train.next_batch(100)

    # compute training values
    if update_train_data:
        a, c, w, b = sess.run([accuracy, cross_entropy, allweights, allbiases], {X: batch_X, Y_: batch_Y})
        print(str(i) + ": accuracy:" + str(a) + " loss: " + str(c) + " (lr:" + str(learning_rate) + ")")

    # compute test values
    if update_test_data:
        a, c = sess.run([accuracy, cross_entropy], test_data)
        print(str(i) + ": ********* epoch " + str(i*100//mnist.train.images.shape[0]+1) 
              + " ********* test accuracy:" + str(a) + " test loss: " + str(c))

    # the backpropagation training step
    sess.run(train_step, {X: batch_X, Y_: batch_Y})

### Run the training

In [ ]:
for i in range(10000+1): training_step(i, i % 100 == 0, i % 20 == 0)
    
print(sess.run(accuracy, feed_dict=test_data))